**This notebook covered by the following [license](License.ipynb)  This note must not be removed**

# Outside we must switch on ingress

`minikube addons enable ingress`

In [5]:
kubectl get pods -A | grep --color=always -E '$|ingress'

NAMESPACE              NAME                                        READY   STATUS      RESTARTS   AGE
kube-scan              kube-scan-db6ffdfd5-zln64                   2/2     Running     0          41m
kube-system            coredns-74ff55c5b-tdzlj                     1/1     Running     0          25h
kube-system            etcd-minikube                               1/1     Running     0          25h
kube-system            ingress-nginx-admission-create-df22b        0/1     Completed   0          40s
kube-system            ingress-nginx-admission-patch-wxsqr         0/1     Completed   0          40s
kube-system            ingress-nginx-controller-558664778f-rnb7s   1/1     Running     0          40s
kube-system            kube-apiserver-minikube                     1/1     Running     0          24h
kube-system            kube-controller-manager-minikube            1/1     Running     0          25h
kube-system            kube-proxy-nsprm                            1/1     Running

In [6]:
kubectl create -f - <<EOF
apiVersion: apps/v1 
kind: Deployment
metadata:
  name: nginx-deployment
  labels:
    app: nginx
spec:
  replicas: 3
  selector:
    matchLabels:
      app: nginx
  template:
    metadata:
      labels:
        app: nginx
    spec:
      containers:
      - name: nginx
        image: nginx
        ports:
        - containerPort: 80
EOF

deployment.apps/nginx-deployment created


In [8]:
kubectl get all

NAME                                    READY   STATUS    RESTARTS   AGE
pod/nginx-deployment-7848d4b86f-55rzd   1/1     Running   0          7s
pod/nginx-deployment-7848d4b86f-l4gbv   1/1     Running   0          7s
pod/nginx-deployment-7848d4b86f-xwrrs   1/1     Running   0          7s

NAME                   TYPE        CLUSTER-IP      EXTERNAL-IP   PORT(S)   AGE
service/kubernetes     ClusterIP   10.96.0.1       <none>        443/TCP   25h
service/nginx-strict   ClusterIP   10.110.68.166   <none>        80/TCP    22h

NAME                               READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/nginx-deployment   3/3     3            3           7s

NAME                                          DESIRED   CURRENT   READY   AGE
replicaset.apps/nginx-deployment-7848d4b86f   3         3         3       7s


In [11]:
kubectl create -f - <<EOF
kind: Service
apiVersion: v1
metadata:
  name: "nginx"
spec:
  selector:
    app: "nginx"
  ports:
    - protocol: "TCP"
      port: 80
      targetPort: 80
  type: LoadBalancer
EOF

service/nginx created


In [12]:
kubectl get svc

NAME         TYPE           CLUSTER-IP       EXTERNAL-IP      PORT(S)        AGE
kubernetes   ClusterIP      10.96.0.1        <none>           443/TCP        25h
nginx        LoadBalancer   10.104.255.186   10.104.255.186   80:31740/TCP   3s


In [13]:
kubectl describe service nginx

Name:                     nginx
Namespace:                default
Labels:                   <none>
Annotations:              <none>
Selector:                 app=nginx
Type:                     LoadBalancer
IP Families:              <none>
IP:                       10.104.255.186
IPs:                      10.104.255.186
LoadBalancer Ingress:     10.104.255.186
Port:                     <unset>  80/TCP
TargetPort:               80/TCP
NodePort:                 <unset>  31740/TCP
Endpoints:                172.17.0.7:80,172.17.0.8:80,172.17.0.9:80
Session Affinity:         None
External Traffic Policy:  Cluster
Events:                   <none>


In [14]:
kubectl get pods --all-namespaces

NAMESPACE              NAME                                        READY   STATUS      RESTARTS   AGE
default                nginx-deployment-7848d4b86f-55rzd           1/1     Running     0          89s
default                nginx-deployment-7848d4b86f-l4gbv           1/1     Running     0          89s
default                nginx-deployment-7848d4b86f-xwrrs           1/1     Running     0          89s
kube-scan              kube-scan-db6ffdfd5-zln64                   2/2     Running     0          43m
kube-system            coredns-74ff55c5b-tdzlj                     1/1     Running     0          25h
kube-system            etcd-minikube                               1/1     Running     0          25h
kube-system            ingress-nginx-admission-create-df22b        0/1     Completed   0          2m32s
kube-system            ingress-nginx-admission-patch-wxsqr         0/1     Completed   0          2m32s
kube-system            ingress-nginx-controller-558664778f-rnb7s   1/1     Run

In [15]:
kubectl get svc --all-namespaces

NAMESPACE              NAME                                 TYPE           CLUSTER-IP       EXTERNAL-IP      PORT(S)                  AGE
default                kubernetes                           ClusterIP      10.96.0.1        <none>           443/TCP                  25h
default                nginx                                LoadBalancer   10.104.255.186   10.104.255.186   80:31740/TCP             39s
kube-scan              kube-scan-ui                         LoadBalancer   10.108.30.142    10.108.30.142    80:30212/TCP             43m
kube-system            ingress-nginx-controller-admission   ClusterIP      10.99.20.76      <none>           443/TCP                  2m38s
kube-system            kube-dns                             ClusterIP      10.96.0.10       <none>           53/UDP,53/TCP,9153/TCP   25h
kubernetes-dashboard   dashboard-metrics-scraper            ClusterIP      10.102.232.89    <none>           8000/TCP                 23h
kubernetes-dashboard   kubernete

In [18]:
kubectl get configmap -n kube-system 

NAME                                 DATA   AGE
coredns                              1      26h
extension-apiserver-authentication   6      26h
ingress-controller-leader-nginx      0      91m
kube-proxy                           2      26h
kube-root-ca.crt                     1      26h
kubeadm-config                       2      26h
kubelet-config-1.20                  1      26h
nginx-load-balancer-conf             1      92m
tcp-services                         0      92m
udp-services                         0      92m


In [19]:
kubectl get configmap -n kube-system  ingress-controller-leader-nginx -o yaml

apiVersion: v1
kind: ConfigMap
metadata:
  annotations:
    control-plane.alpha.kubernetes.io/leader: '{"holderIdentity":"ingress-nginx-controller-558664778f-rnb7s","leaseDurationSeconds":30,"acquireTime":"2021-01-24T16:03:31Z","renewTime":"2021-01-24T17:35:29Z","leaderTransitions":0}'
  creationTimestamp: "2021-01-24T16:03:31Z"
  managedFields:
  - apiVersion: v1
    fieldsType: FieldsV1
    fieldsV1:
      f:metadata:
        f:annotations:
          .: {}
          f:control-plane.alpha.kubernetes.io/leader: {}
    manager: nginx-ingress-controller
    operation: Update
    time: "2021-01-24T16:03:31Z"
  name: ingress-controller-leader-nginx
  namespace: kube-system
  resourceVersion: "69626"
  uid: ae9f3f93-e329-43bb-9d58-85d815326eb1


In [20]:
kubectl get configmap -n kube-system  nginx-load-balancer-conf -o yaml

apiVersion: v1
data:
  hsts: "false"
kind: ConfigMap
metadata:
  annotations:
    kubectl.kubernetes.io/last-applied-configuration: |
      {"apiVersion":"v1","data":{"hsts":"false"},"kind":"ConfigMap","metadata":{"annotations":{},"labels":{"addonmanager.kubernetes.io/mode":"EnsureExists"},"name":"nginx-load-balancer-conf","namespace":"kube-system"}}
  creationTimestamp: "2021-01-24T16:03:07Z"
  labels:
    addonmanager.kubernetes.io/mode: EnsureExists
  managedFields:
  - apiVersion: v1
    fieldsType: FieldsV1
    fieldsV1:
      f:data:
        .: {}
        f:hsts: {}
      f:metadata:
        f:annotations:
          .: {}
          f:kubectl.kubernetes.io/last-applied-configuration: {}
        f:labels:
          .: {}
          f:addonmanager.kubernetes.io/mode: {}
    manager: kubectl-client-side-apply
    operation: Update
    time: "2021-01-24T16:03:07Z"
  name: nginx-load-balancer-conf
  namespace: kube-system
  resourceVersion: "64667"
  uid: 489dd253-d744-41f7-9e76-62d458c